Multisent using Naive bayes

In [1]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import xgboost, numpy, string
import pandas as pd
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

#출처
#https://www.analyticsvidhya.com/blog/2018/04/a-comprehensive-guide-to-understand-and-implement-text-classification-in-python/

Using TensorFlow backend.


In [14]:
#train, test, valid 데이터 read
#각 데이터는 xxx파일에 저장되어 있습니다.
#happy의 경우 happy1만을 세팅한 값입니다. 추후 변경 예정
train_data = pd.read_csv("four_train_data.csv")
test_data = pd.read_csv("four_test_data.csv")
valid_data = pd.read_csv("four_valid_data.csv")

In [15]:
#잘 읽었는지 확인하기
#train_data.head()
#test_data.head()
valid_data.head()

,text,label
0,어떻게 각색을 하는데 더구리게\r\n할수가 있냐....\r\n신선한건 다쳐내고 진부...,sad
1,뮤지컬 엑스칼리버 DK 도겸 회차 양도 \r\n\r\n6/25(화) 7시 1층 B구...,happy
2,피치\r\n 사랑의 멋짐을 모르는 당신이 불쌍해요!라는 명대사가 있지요 웨딩피치=사...,fear
3,아 맞 다 그 랫 지 수 술 할 정 도 라 고 ? !,fear
4,어머... 그래요,fear


In [16]:
# split the dataset into training and validation datasets 
train_x, train_y = train_data['text'], train_data['label']
valid_x, valid_y = valid_data['text'], valid_data['label']
# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [18]:
#count_vect.fit(train_data['text']) 에러 나는데 같이 풀어야 할 것 같네요....
#ValueError: np.nan is an invalid document, expected byte or unicode string.
#스택 오버 플로우에 하라는데로 했는데도 안됨....
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(train_data['text'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

ValueError: np.nan is an invalid document, expected byte or unicode string.

이 뒤로는 아직 실행 안시키고 
붙여 넣기만 한거...
위에께 안돼서 못하는중

In [35]:
#Stopwords 만들게요....
stop_words_set = pd.read_csv('stopwords100.txt',header = 0, delimiter = '\t', quoting = 3)
stop_words= (list(stop_words_set['aa']))
stop_words2 = ['은', '는', '이', '가', '하', '아', '것', '들','의', '있', '되', '수', '보', '주', '등', '한']
stop_words.extend(stop_words2)

In [37]:
#토크나이저 만들게요.... 책 참고.
from soynlp.tokenizer import MaxScoreTokenizer
from soynlp.normalizer import *
import re
from konlpy.tag import Okt

def preprocessing(text, okt, remove_stopwords = False, stop_words = []):
    text = only_hangle(text)
    text = repeat_normalize(text, num_repeats = 2)
    
    text_token = okt.morphs(text, stem = True)
    
    if remove_stopwords:
        text_token = [token for token in text_token if not token in stop_words]
        
    return text_token

In [41]:
#Train data & Valid data를 전처리 하겠습니다
#책에서 나온거 그대로
okt = Okt()
clean_train_data = []
clean_valid_data = []
clean_test_data = []
for text in train_data['text']:
    if type(text) == str:
        clean_train_data.append(preprocessing(text, okt, remove_stopwords = True, stop_words = stop_words))
    else:
        clean_train_data.append([])
        
for text in valid_data['text']:
    if type(text) == str:
        clean_valid_data.append(preprocessing(text, okt, remove_stopwords = True, stop_words = stop_words))
    else:
        clean_valid_data.append([])
        
for text in test_data['text']:
    if type(text) == str:
        clean_test_data.append(preprocessing(text, okt, remove_stopwords = True, stop_words = stop_words))
    else:
        clean_test_data.append([])

In [51]:
#잘됐나 테스트 해볼게요
clean_train_data[:4]
#clean_valid_data[:4]
#clean_test_data[:4]

[['집안', '어른', '동성동본', '결혼', '하다', '족보', '를', '갈아엎다', '등본', '떼다', '전주최씨', '자다'],
 ['진짜', '많이', '빠지다', 'ㅠㅠ', '아이', '홀', '라인', '거의', '쌍꺼풀', '돼다'],
 ['알다'],
 ['음', '아마', '아침', '드라마', '에서', '보다', '배우', '장', '선영', '이다', '배우', '미소']]

In [52]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_train_data)
train_sequences = tokenizer.texts_to_sequences(clean_train_data)
valid_sequences = tokenizer.texts_to_sequences(clean_valid_data)
test_sequences = tokenizer.texts_to_sequences(clean_test_data)

word_index = tokenizer.word_index

MAX_SEQUENCE_LENGTH = 70

In [54]:
#워드 임베딩으로 조져 볼게요
embeddings_index = {}
for i, line in enumerate(open('wiki.ko.vec')):
    values = line.split()
    embeddings_index[values[0]] = numpy.asarray(values[1:], dtype='float32')
    
    
embedding_matrix = numpy.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector    

In [57]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)

    return metrics.accuracy_score(predictions, valid_y)

In [81]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(train_data['text'],train_data['label'],random_state=0)

vect = CountVectorizer().fit(X_train)
X_train_vectorized = vect.transform(X_train)
clfrNB = naive_bayes.MultinomialNB(alpha = 0.1)
clfrNB.fit(X_train_vectorized, y_train)
preds = clfrNB.predict(vect.transform(X_test))

ValueError: np.nan is an invalid document, expected byte or unicode string.